In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
from datetime import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
# Setup Database
# Create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
# Reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)
# Save references to each table
measurements = Base.classes.measurement
stations = Base.classes.station

# Query 1: Precipitation

In [ ]:
# Create session and query
session = Session(engine)
resultss = session.query(measurements.date, measurements.prcp).all()
results = []
# Drop 'None' values
for result in resultss:
    if result[1] != None :
        results.append(result)
results

In [62]:
# Create dictionary
precdates = []
for date in results:
    if date[0] not in precdates:
        precdates.append(date[0])
precdict = dict.fromkeys(precdates,0)
for prdate in precdates:
    precdict[str(prdate)] = []
    for date in results:
        if date[0] == prdate:
            precdict[str(prdate)].append(date[1])
precdict

{'2010-01-01': [0.08, 0.28, 0.21, 0.15, 0.15, 0.05, 0.14],
 '2010-01-02': [0.0, 0.0, 0.02, 0.0, 0.0, 0.0, 0.0],
 '2010-01-03': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '2010-01-04': [0.0, 0.0, 0.01, 0.0, 0.0, 0.0, 0.0],
 '2010-01-07': [0.06, 0.38, 0.7, 0.17, 0.04, 0.3],
 '2010-01-08': [0.0, 0.01, 0.03, 0.02, 0.0, 0.0, 0.0],
 '2010-01-09': [0.0, 0.0, 0.01, 0.01, 0.0, 0.0, 0.0],
 '2010-01-10': [0.0, 0.0, 0.01, 0.0, 0.0, 0.0, 0.01],
 '2010-01-11': [0.01, 0.03, 0.14, 0.48, 0.03, 0.05, 0.14],
 '2010-01-12': [0.0, 0.0, 0.02, 0.0, 0.0, 0.0, 0.0],
 '2010-01-14': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '2010-01-15': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '2010-01-16': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '2010-01-17': [0.0, 0.0, 0.0, 0.0, 0.0],
 '2010-01-18': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '2010-01-19': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '2010-01-20': [0.0, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0],
 '2010-01-21': [0.0, 0.04, 0.13, 0.16, 0.0, 0.32],
 '2010-01-22': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '2010-01-23': [0.0

# Query 2: Stations

In [65]:
# Create session and query
session = Session(engine)
results = session.query(measurements.station,stations.id).\
    filter(measurements.station == stations.station).\
    group_by(measurements.station).all()
results

[('USC00511918', 8),
 ('USC00513117', 2),
 ('USC00514830', 3),
 ('USC00516128', 9),
 ('USC00517948', 4),
 ('USC00518838', 5),
 ('USC00519281', 7),
 ('USC00519397', 1),
 ('USC00519523', 6)]

In [66]:
# Get a list of the stations 
statnames = []
for result in results:
    statnames.append(result[0])
statnames

['USC00511918',
 'USC00513117',
 'USC00514830',
 'USC00516128',
 'USC00517948',
 'USC00518838',
 'USC00519281',
 'USC00519397',
 'USC00519523']

# Query 3: Temperatures

In [69]:
# Design a query to find the most active station
actstation = session.query(measurements.station,stations.id,func.count(measurements.station)).\
    filter(measurements.station == stations.station).\
    group_by(measurements.station).order_by(func.count(measurements.station).desc()).first()
actstation

('USC00519281', 7, 2772)

In [70]:
# Get the id for the most active station
actid = actstation[1]
actid

7

In [72]:
# Using the most active station id, get the most recent measurement
recdate = session.query(measurements).\
    filter(measurements.station == stations.station).\
    filter(stations.id == int(actid)).\
    order_by(measurements.date.desc()).first()
recdate = recdate.date
recdate

'2017-08-18'

In [76]:
# Starting from the most recent data point in the database. 
date0 = dt.strptime(recdate, '%Y-%m-%d')
# Calculate the date one year from the last date in
yearago = date0.year - 1
date1 = f'{yearago}-{date0.month}-{date0.day}'
date1 = dt.strptime(date1, '%Y-%m-%d')
date1

datetime.datetime(2016, 8, 18, 0, 0)

In [79]:
# Create query and obtain data
precactive_q = session.query(stations.id, measurements.station, measurements.date, measurements.tobs).\
    filter(measurements.station == stations.station).\
    filter(measurements.date > date1.strftime('%Y-%m-%d')).\
    filter(stations.id == int(actid)).\
    order_by(measurements.date).all()
precactive_q

[(7, 'USC00519281', '2016-08-19', 79.0),
 (7, 'USC00519281', '2016-08-20', 81.0),
 (7, 'USC00519281', '2016-08-21', 79.0),
 (7, 'USC00519281', '2016-08-22', 78.0),
 (7, 'USC00519281', '2016-08-23', 77.0),
 (7, 'USC00519281', '2016-08-24', 77.0),
 (7, 'USC00519281', '2016-08-25', 80.0),
 (7, 'USC00519281', '2016-08-26', 80.0),
 (7, 'USC00519281', '2016-08-27', 75.0),
 (7, 'USC00519281', '2016-08-28', 73.0),
 (7, 'USC00519281', '2016-08-29', 78.0),
 (7, 'USC00519281', '2016-08-30', 77.0),
 (7, 'USC00519281', '2016-08-31', 78.0),
 (7, 'USC00519281', '2016-09-01', 80.0),
 (7, 'USC00519281', '2016-09-02', 80.0),
 (7, 'USC00519281', '2016-09-03', 78.0),
 (7, 'USC00519281', '2016-09-04', 78.0),
 (7, 'USC00519281', '2016-09-05', 78.0),
 (7, 'USC00519281', '2016-09-06', 73.0),
 (7, 'USC00519281', '2016-09-07', 74.0),
 (7, 'USC00519281', '2016-09-08', 80.0),
 (7, 'USC00519281', '2016-09-09', 79.0),
 (7, 'USC00519281', '2016-09-10', 77.0),
 (7, 'USC00519281', '2016-09-11', 80.0),
 (7, 'USC0051928

In [108]:
# Create dictionary with information
tempdates = []
temptemps = []
for date in precactive_q:
    if date[2] not in tempdates:
        tempdates.append(date[2])
    temptemps.append(date[3])
tempdict = dict.fromkeys(tempdates, 0)
counter = 0
for date in tempdates:
    tempdict[str(date)] = tempdict[str(date)] + temptemps[int(counter)]
    counter = counter + 1
tempdics = {"Station ID" : precactive_q[0][0], "Station Name" : precactive_q[0][1], "Measurements" : tempdict}
tempdics

{'Station ID': 7,
 'Station Name': 'USC00519281',
 'Measurements': {'2016-08-19': 79.0,
  '2016-08-20': 81.0,
  '2016-08-21': 79.0,
  '2016-08-22': 78.0,
  '2016-08-23': 77.0,
  '2016-08-24': 77.0,
  '2016-08-25': 80.0,
  '2016-08-26': 80.0,
  '2016-08-27': 75.0,
  '2016-08-28': 73.0,
  '2016-08-29': 78.0,
  '2016-08-30': 77.0,
  '2016-08-31': 78.0,
  '2016-09-01': 80.0,
  '2016-09-02': 80.0,
  '2016-09-03': 78.0,
  '2016-09-04': 78.0,
  '2016-09-05': 78.0,
  '2016-09-06': 73.0,
  '2016-09-07': 74.0,
  '2016-09-08': 80.0,
  '2016-09-09': 79.0,
  '2016-09-10': 77.0,
  '2016-09-11': 80.0,
  '2016-09-12': 76.0,
  '2016-09-13': 79.0,
  '2016-09-14': 75.0,
  '2016-09-15': 79.0,
  '2016-09-16': 78.0,
  '2016-09-17': 79.0,
  '2016-09-18': 78.0,
  '2016-09-19': 78.0,
  '2016-09-20': 76.0,
  '2016-09-21': 74.0,
  '2016-09-22': 77.0,
  '2016-09-23': 78.0,
  '2016-09-24': 79.0,
  '2016-09-25': 79.0,
  '2016-09-26': 77.0,
  '2016-09-27': 80.0,
  '2016-09-28': 78.0,
  '2016-09-29': 78.0,
  '2016-09

# Query 4: Start

In [ ]:
# Close session
session.close()